In [1]:
import numpy as np
import numbers
import time
from abc import ABCMeta, abstractmethod
from typing import TypeVar, Union, Any
from random import randint
import matplotlib.pyplot as plt
Num = TypeVar('Num', int, float)

In [2]:
class CreditCard:
    """A consumer credit card"""
    
    def __init__(self, customer, bank, acnt, limit):
        """Create a new credit card instance.
        
        The initial balance is zero.
        
        :param customer: the name of the customer 
        :param bank: the name of the bank
        :param acnt: the account identifier
        :param limit: credit limit
        """
        
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0
        
    def get_customer(self):
        """Return name of the customer"""
        return self._customer
    
    def get_bank(self):
        """Return the bank's name."""
        return self._bank
    
    def get_account(self):
        """Return the card identifying number (typically stored as a string."""
        return self._account
    
    def get_limit(self):
        """Return current credit limit."""
        return self._limit
    
    def get_balance(self):
        """Return current balance."""
        return self._balance
    
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed; False if charge was denied.
        """
        
        if price + self._balance > self._limit:
            return False
        else:
            self._balance += price
            return True
    
    def make_payment(self, amount):
        """Process customer payment that reduces balance."""
        self._balance -= amount 

Python interpreter automatically binds the instance upon which the method is invoked to the self parameter.

In [3]:
cc = CreditCard('John Doe', 'Bank', '1234 5678', 1000)
cc.get_limit()

1000

In class, __init__ method works as the constructor of the class. Also, single leading underscore in the name of a data member, such as _balance, implies that it is intended as nonpublic. Users of a class should not directly access such members.

For better encapsulation, it is mostly better to treat all data members as nonpublic and provide accessors, to provide a user of our class read-only access to a trait, and update methods for updating its memebers.

# Reinforcement

### *R-2.4*
Write a Python class, Flower, that has three instance variables of type str, int, and float, that respectively represent the name of the flower, its number of petals, and its price. Your class must include a constructor method that initializes each variable to an appropriate value, and your class should include methods for setting the value of each type, and retrieving the value of each type.

In [4]:
class Flower:
    def __init__(self, name, n_petals, price):
        self._name = name
        self._n_petals = n_petals
        self._price = price
        
    def get_name(self):
        return self._name
    
    def set_name(self, name):
        self._name = name
        
    def get_n_petals(self):
        return self._n_petals
    
    def set_n_petals(self, n_petals):
        self._n_petals = n_petals
        
    def get_price(self):
        return self._price
    
    def set_price(self, price):
        self._price = price

### *R-2.5, 2.6, 2.7*
R-2.5: Use the techniques of Section 1.7 to revise the charge and make payment methods of the CreditCard class to ensure that the caller sends a number as a parameter.

R-2.6: If the parameter to the make payment method of the CreditCard class were a negative number, that would have the effect of raising the balance on the account. Revise the implementation so that it raises a ValueError if a negative value is sent.

R-2.7: The CreditCard class of Section 2.3 initializes the balance of a new ac- count to zero. Modify that class so that a new account can be given a nonzero balance using an optional fifth parameter to the constructor. The four-parameter constructor syntax should continue to produce an account with zero balance.

In [5]:
class CreditCard:
    def __init__(self, customer, bank, account, limit, balance=0):
        self._customer = customer
        self._bank = bank
        self._account = account
        self._limit = limit
        self._balance = balance
        
    def charge(self, price):
        '''Charge given price to the card, aasuming sufficient credit limit.
        Return True if charge was processed; False if charge was denied'''
        try:
            assert isinstance(price, (int, float, complex))
        except AssertionError:
            print("The price must be a number!")
            #exit the function
            return
        
        # if charge exceed limit
        if price + self._balance > self._limit:
            return False
        else:
            self._balance += price
            return True
        
    def make_payment(self, amount):
        try:
            assert isinstance(amount, numbers.Number)
        except AssertionError:
            print('The amount must be a number!')
            # exit the function
            return
        # is positive or not
        if amount < 0:
            raise ValueError('amount must be a positive number')
        '''Process customer payment that reduces balance'''
        self._balance -= amount

### *R-2.9 -> R-2.15*

In [6]:
class Vector:
    def __init__(self, data):
        if isinstance(data, int):
            self._data = [0] * data
            self._len = data
        elif isinstance(data, list):
            self._data = data
            self._len = len(data)
        else:
            raise ValueError("Vector must be initialize by a int or a list.")
            
            
    def __repr__(self):
        return repr(self._data).replace('[', '<').replace(']', '>')
    
    def __len__(self):
        return self._len
    
    def __getitem__(self, index: int):
        return self._data[index]
    
    def __setitem__(self, index: int, value: Num):
        self._data[index] = value
        
    def __add__(self, v: 'Vector'):
        '''
        >>> v1 = Vector([1, 2, 3])
        >>> v2 = Vector([1, 1, 1])
        >>> v1 + v2
        <2, 3, 4>
        '''
        assert len(self) == len(v)
        result = Vector(self._len)
        for i in range(len(self)):
            result[i] = self._data[i] + v[i]
        return result
    
    def __radd__(self, v: 'Vector'):
        return self.__add__(v)
    
    def __sub__(self, v: 'Vector'):
        """
        >>> v1 = Vector([1, 2, 3])
        >>> v2 = Vector([1, 1, 1])
        >>> v1 - v2
        <0, 1, 2>
        """
        assert len(self) == len(v)
        result = Vector(self._len)
        for i in range(len(self)):
            result[i] = self._data[i] - v[i]
        return result
    
    def __neg__(self):
        """
        >>> -Vector([1, 2, 3])
        <-1, -2, -3>
        """
        result = Vector(self._len)
        for index, value in enumerate(self._data):
            result[index] = -value
        return result
    
    def __mul__(self, factor: Union[int, 'Vector']):
        """multiply with an int or vector.
        >>> v1 = Vector([1, 2, 3])
        >>> v1 * 2
        <2, 4, 6>
        >>> 2 * v1
        <2, 4, 6>
        >>> v2 = Vector([1, 1, 1])
        >>> v1 * v2
        6
        """
        if isinstance(factor, Vector):
            assert len(self) == len(factor)
            result = 0
            for i in range(self._len):
                result += self._data[i] * factor[i]
            return result
        else:
            result = Vector(self._len)
            for index, value in enumerate(self._data):
                result[index] = factor * value
            return result
        
    def __rmul__(self, factor: Union[int, 'Vector']):
        return self.__mul__(factor)

In [7]:
x = Vector(5)
x[2] = 10
x

<0, 0, 10, 0, 0>

In [8]:
x + x

<0, 0, 20, 0, 0>

In [9]:
x * x

100

### *R-2.18*
Give a short fragment of Python code that uses the progression classes from Section 2.4.2 to find the 8 th value of a Fibonacci progression that starts with 2 and 2 as its first two values.

In [10]:
class Progression:
    def __init__(self, start=0):
        self._current = start
        
    def _advance(self):
        self._current += 1
        
    def __next__(self):
        if self._current is None:
            raise StopIteration
        else:
            answer = self._current
            self._advance()
            return answer
        
    def __iter__(self):
        return self
    
    def print_progression(self, n):
        print(' '.join(str(next(self)) for i in range(n)))
        
        
class FibonacciProgression(Progression):
    def __init__(self, first=0, second=1):
        # initialize base class
        # start progression at first
        super().__init__(first)
        self._prev = second - first
        
    def _advance(self):
        self._prev, self._current = self._current, self._current + self._prev

In [11]:
fib = FibonacciProgression(2, 2)
fib.print_progression(8)

2 2 4 6 10 16 26 42


### *R-2.19*
When using the ArithmeticProgression class of Section 2.4.2 with an increment of 128 and a start of 0, how many calls to next can we make before we reach an integer of 2^63 or larger?

In [12]:
steps = 2 ** (63 - 7)
steps

72057594037927936

# Creativity

### *C-2.26*
The SequenceIterator class of Section 2.3.4 provides what is known as a forward iterator. Implement a class named ReversedSequenceIterator that serves as a reverse iterator for any Python sequence type. The first call to next should return the last element of the sequence, the second call to next should return the second-to-last element, and so forth.

In [13]:
class ReversedSequenceIterator:
    def __init__(self, sequence):
        self._seq = sequence
        self._k = len(sequence)
        
    def __next__(self):
        self._k -= 1
        if self._k >= 0:
            return self._seq[self._k]
        else:
            raise StopIteration
            
    def __iter__(self):
        return self

In [14]:
rev_seq = ReversedSequenceIterator([1, 3, 5])

for i in rev_seq:
    print(i)

5
3
1


### *C-2.31*
Write a Python class that extends the Progression class so that each value in the progression is the absolute value of the difference between the previous two values. You should include a constructor that accepts a pair of numbers as the first two values, using 2 and 200 as the defaults.

In [15]:
class AbsdiffPrograssion(Progression):
    def __init__(self, first=2, second=200):
        super().__init__(first)
        self._prev1 = None
        self._prev2 = None
        # for starting
        self._second = second
        self._count = 1

    def _advance(self):
        # for starting
        if self._count == 1:
            self._prev1 = self._current
            self._current = self._second
        else:
            self._prev1, self._prev2 = self._current, self._prev1
            self._current = abs(self._prev1 - self._prev2)
        self._count += 1

In [16]:
t = AbsdiffPrograssion()
t.print_progression(10)

2 200 198 2 196 194 2 192 190 2
